# Monitoring an ongoing MIBI run

This notebook can be run alongside an active MIBIScope run.  As images are generated, this notebook will automatically pass the data through pre-specified functions, like tiff 
extraction, qc metric computation, etc. Eventually, all of the processing steps in the toffy repo will be incorporated into the Watcher functionality here. For now, this notebook will automatically extract tifs and generate QC plots of your data. 

## This notebook is an example: create a copy before running it or you will get merge conflicts!

In [ ]:
import sys
sys.path.append('../')

import os
import pandas as pd

from toffy.fov_watcher import start_watcher
from toffy.watcher_callbacks import build_callbacks

## Required variables

 - `run_name` should contain the exact name of the MIBI run that will be monitored. 
 - `panel_path` should point to a panel csv specifying the targets on your panel. For an example, see the [example panel file](https://github.com/angelolab/toffy/blob/main/files/example_panel_file.csv)

In [ ]:
# the name of the run
run_name = 'YYYY-MM-DD_run_name'

# path to user panel
panel_path = 'C:\\Users\\Customer.ION\\Documents\\panel_files\\my_cool_panel.csv'


# these are set automatically
base_dir = os.path.join('D:\\Data', run_name)

# path to log folder
log_path = os.path.join('C:\\Users\\Customer.ION\\Documents\\watcher_logs', run_name)

# load panel
panel = pd.read_csv(panel_path)

## Callback initialization

Callbacks listed in the `run_callbacks` argument will be run only once all expected FoV's have been
discovered and processed. Callbacks listed in `fov_callbacks` arugment will be run on each
completed FoV, the moment both the .bin & .json are found.

Some run callbacks depend on certain fov callbacks.  You do not need to manually set these, as
setting a run callback will automatically set the prerequisite fov callbacks to run.

Each callback will need a location to save its relevant data. These are passed as *_out_dir
arguments to the `build_callbacks` function.  If required arguments are not set by the user,
this cell will throw an error and list the missing requireed arguments.

The `extract_tiffs` fov callback specifies that every FoV generate tiffs according to the supplied
`panel`. Additional arguments are `intensities` which when set to True, will extract the intensity
data, and `time_res` which will overwrite the default time resolution.

The `plot_qc_metrics` run callback will run all currently available qc metrics on each FoV, and 
plot the results once the run has completed. Additional arguments are:
    - `gaussian_blur`
    - `blur_factor`
    - `axes_size`
    - `wrap`
    - `dpi`
    - `save_dir`
    
The `plot_mph_metrics` run callback will compute the median pulse height data for each 
FoV, and plot the results once the run has completed. Additional arguments are:
`regression` which when set to True will also plot the linear regression line for the data.

In [ ]:
extraction_dir = os.path.join('D:\\Extracted_Images', run_name)
qc_dir = os.path.join('C:\\Users\\Customer.ION\\Documents\\run_metrics', run_name)

fov_callback, run_callback = build_callbacks(
    run_callbacks = ['plot_qc_metrics', 'plot_mph_metrics'],
    fov_callbacks = ['extract_tiffs'],
    tiff_out_dir=extraction_dir,
    qc_out_dir=qc_dir,
    mph_out_dir=qc_dir,
    plot_dir=qc_dir,
    panel=panel,
    intensities=False,
    regression=False,
)

In [ ]:
start_watcher(base_dir, log_path, fov_callback, run_callback)